In [1]:
import os, sys, time

import requests
import shutil

# https://gitlab.com/dlr-ve-esy/esy-osmfilter/-/tree/master/
from esy.osmfilter import  osm_colors          as CC
from esy.osmfilter import run_filter, Node,Way,Relation 
from esy.osmfilter import export_geojson
from contextlib import contextmanager

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
# import logging
# logging.basicConfig()
# logger=logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
# logger.setLevel(logging.WARNING)


In [6]:
country_name = 'africa' # TODO: Create dictionary with country_code : country_name pairs
country_code = 'AFRICA'
# country_name = 'cape-verde'
geofabrik_filename=f'{country_name}-latest.osm.pbf' # Filename for geofabrik 
geofabrik_url = f'https://download.geofabrik.de/africa/{geofabrik_filename}' # https://download.geofabrik.de/africa/nigeria-latest.osm.pbf

local_file= os.path.join("data","osm",geofabrik_filename)
if not os.path.exists(local_file):
    print(f"{geofabrik_filename} does not exist, downloading to {local_file}")
    os.makedirs(os.path.dirname(local_file), exist_ok=True) # create data/osm directory
    with requests.get(geofabrik_url, stream=True) as r:
        with open(local_file, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

# Input filepath
PBF_inputfile = os.path.join(os.getcwd(),local_file)

# json file for the Data dictionary 
# json file for the Elements dictionary is automatically written to 'data/osm/Elements'+filename)
JSON_outputfile = os.path.join(os.getcwd(),'data','osm',country_name+'_substations.json')



In [30]:
print(PBF_inputfile, JSON_outputfile)

/home/matin/pypsa-africa/data_exploration/data/osm/africa-latest.osm.pbf /home/matin/pypsa-africa/data_exploration/data/osm/africa_substations.json


In [31]:
#see https://dlr-ve-esy.gitlab.io/esy-osmfilter/filter.html for filter structures

prefilter = {Node: {"power":["substation"]}, Way: {"power":["substation"]}, Relation: {"power":["substation"]}}
blackfilter = [("pipeline","substation"),]
whitefilter =[[("power","substation"),],]
    
elementname=f'{country_code}_substations'

[Data,Elements]=run_filter(elementname,PBF_inputfile, JSON_outputfile,prefilter,whitefilter,blackfilter, NewPreFilterData=False, CreateElements=False, LoadElements=True,verbose=False,multiprocess=True)


# export the elements to a geojson-file
# export_geojson(Elements[elementname]['Way'],Data,filename='test.geojson',jsontype='Line')
# convert extract node elements
# export_geojson(Elements[elementname]['Node'],Data,filename='test.geojson',jsontype='Point')




In [9]:
import pickle
Elements = pickle.load(os.path.join(os.getcwd(),os.path.dirname(JSON_outputfile), 'Elements', 'AFRICA_substations.pickle'))

# Elements = osm_pickle.pickleload(Elements,os.path.join(os.getcwd(),os.path.dirname(JSON_outputfile), 'Elements/'))    

TypeError: file must have 'read' and 'readline' attributes

In [33]:
df = pd.json_normalize(Elements[elementname]["Way"].values())
df = df.dropna(subset=['tags.voltage'])
df = df.dropna(axis=1, how='all')

display(df)



,id,refs,tags.fixme,tags.power,tags.access,tags.barrier,tags.voltage,tags.location,tags.substation,tags.height,...,tags.brand:wikipedia,tags.note,tags.description,tags.gas_insulated,tags.website,tags.start_date,tags.generator:output:electricity,tags.owner,tags.addr:suburb,tags.operator:fr
0,90137992,"[1044958475, 1044958478, 1044958443, 104495847...","name,ref,operator,voltage,frequency",substation,no,fence,66000,outdoor,distribution,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,219596292,"[2287444706, 2287444696, 2287444709, 228744469...",NaN,substation,NaN,NaN,88000,outdoor,distribution,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,694348420,"[6518987551, 6518987549, 6518987548, 651898755...",NaN,substation,NaN,NaN,110000,outdoor,industrial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,302904657,"[3071892870, 3071892871, 3071892873, 307189287...",NaN,substation,NaN,fence,110000,outdoor,transmission,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,90454383,"[1049492731, 1049492535, 1049493108, 104949262...",NaN,substation,NaN,NaN,400000;220000;132000,outdoor,transmission,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6818,223115963,"[2320515555, 2320515539, 2320515553, 232051555...",NaN,substation,NaN,fence,220000,outdoor,transmission,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6819,302127350,"[3063706734, 3063706732, 3063706728, 306370673...",NaN,substation,NaN,NaN,161000,outdoor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6822,695523467,"[6530697089, 6530697088, 6530697087, 653069708...",NaN,substation,NaN,NaN,66000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6824,204050015,"[2140628605, 2140628602, 2140628606, 214062860...",NaN,substation,NaN,NaN,132000,outdoor,distribution,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df1 = pd.json_normalize(Elements[elementname]["Node"].values())
df1 = df1.dropna(axis=1, how='all')

display(df1)


,id,lonlat,tags.power,tags.operator,tags.ref,tags.substation,tags.name,tags.source,tags.designation,tags.addr:city,...,tags.man_made,tags.street_cabinet,tags.disused,tags.building,tags.barrier,tags.branch,tags.notes,tags.operator_type,tags.FIXME,tags.operator:type
0,4967361644,"[-14.336955699999914, 28.063764700000114]",substation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5293654156,"[16.206542499999966, -20.8426169]",substation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5291355777,"[24.52657030000007, -17.727550399999974]",substation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2466505236,"[-16.381263900000057, 28.534258600000015]",substation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000011529,"[5.580595200000045, 6.357239900000007]",substation,Transformer,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,2539977618,"[-16.66755369999997, 28.051217699999974]",substation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
521,5840061103,"[-25.309837200000064, 16.95873199999999]",substation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522,1983304051,"[22.028369600000204, -33.22827809999993]",substation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
523,3033268054,"[3.229681700000007, 36.72909620000005]",substation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
lonlat_column =[]
for ref in df["refs"]:
    lonlats =[]
    for r in ref:
        lonlat = Data["Node"][str(r)]["lonlat"]
        lonlats.append(lonlat)
    lonlats = np.array(lonlats)
    lonlat = np.mean(lonlats, axis=0)
    lonlat_column.append(lonlat)
# display(lonlat_column)
df["lonlat"] = lonlat_column

display(df)
    
# polygon_column=[]
# for poly_list in lonlat_column:
#     for p in poly_list:
#         print(p)

#     print("p")
    

,id,refs,tags.fixme,tags.power,tags.access,tags.barrier,tags.voltage,tags.location,tags.substation,tags.height,...,tags.note,tags.description,tags.gas_insulated,tags.website,tags.start_date,tags.generator:output:electricity,tags.owner,tags.addr:suburb,tags.operator:fr,lonlat
0,90137992,"[1044958475, 1044958478, 1044958443, 104495847...","name,ref,operator,voltage,frequency",substation,no,fence,66000,outdoor,distribution,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[27.793264257142766, -32.982510257142984]"
2,219596292,"[2287444706, 2287444696, 2287444709, 228744469...",NaN,substation,NaN,NaN,88000,outdoor,distribution,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[30.68760782857152, -27.96400892857118]"
5,694348420,"[6518987551, 6518987549, 6518987548, 651898755...",NaN,substation,NaN,NaN,110000,outdoor,industrial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[13.8535432200001, -4.144742079999883]"
6,302904657,"[3071892870, 3071892871, 3071892873, 307189287...",NaN,substation,NaN,fence,110000,outdoor,transmission,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[13.289175300000187, -4.1735521200000125]"
9,90454383,"[1049492731, 1049492535, 1049493108, 104949262...",NaN,substation,NaN,NaN,400000;220000;132000,outdoor,transmission,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[25.632686079999928, -33.71928261999984]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6818,223115963,"[2320515555, 2320515539, 2320515553, 232051555...",NaN,substation,NaN,fence,220000,outdoor,transmission,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[31.42202072857132, 21.774249871428687]"
6819,302127350,"[3063706734, 3063706732, 3063706728, 306370673...",NaN,substation,NaN,NaN,161000,outdoor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[-0.36905076249995517, 6.209410687499963]"
6822,695523467,"[6530697089, 6530697088, 6530697087, 653069708...",NaN,substation,NaN,NaN,66000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[4.470365639999967, 35.703675340000025]"
6824,204050015,"[2140628605, 2140628602, 2140628606, 214062860...",NaN,substation,NaN,NaN,132000,outdoor,distribution,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[26.83819403999998, -27.95507242000005]"


In [37]:
gdf = gpd.GeoDataFrame(f, geometry=[Point(x, y) for x, y in df.lonlat])

SyntaxError: unexpected EOF while parsing (<ipython-input-37-5e7ee8241d35>, line 1)

In [29]:
display(gdf)

,id,refs,tags.power,tags.voltage,tags.location,tags.substation,tags.fixme,tags.barrier,tags.name,lonlat,geometry
0,669957212,"[6274221433, 6274221434, 6274221435, 627422143...",substation,330000;132000,outdoor,transmission,NaN,NaN,NaN,"[3.6143597400000296, 6.565371219999987]",POINT (3.61436 6.56537)
1,563715917,"[5432096542, 5432096541, 5432096540, 543209653...",substation,330000,outdoor,transmission,NaN,NaN,NaN,"[6.629980380000002, 7.822560159999988]",POINT (6.62998 7.82256)
2,477596994,"[4709180541, 4709180540, 4709180539, 470918053...",substation,330000;132000,outdoor,transmission,name,NaN,NaN,"[4.577366299999998, 7.80037873333333]",POINT (4.57737 7.80038)
9,565866986,"[5448653454, 5448653453, 5448653452, 544865344...",substation,330000,outdoor,transmission,name,NaN,NaN,"[4.708973323076923, 6.7342855230769185]",POINT (4.70897 6.73429)
12,746582404,"[6986702202, 6986702201, 6986702200, 698670219...",substation,330000,outdoor,transmission,NaN,NaN,NaN,"[7.15676494999997, 4.762162887499925]",POINT (7.15676 4.76216)
...,...,...,...,...,...,...,...,...,...,...,...
153,395182307,"[3980844354, 3980844347, 3980844352, 398084436...",substation,132000,outdoor,transmission,NaN,NaN,NaN,"[9.348810479999997, 11.689970420000018]",POINT (9.34881 11.68997)
160,215459565,"[2249105625, 2258352498, 2249105563, 225835251...",substation,330000;132000,outdoor,transmission,NaN,fence,NaN,"[6.654831054545443, 7.471823409090908]",POINT (6.65483 7.47182)
163,563720436,"[5432144602, 5432144601, 5432144600, 543214459...",substation,132000,outdoor,transmission,NaN,NaN,NaN,"[8.4566884, 11.873461219999996]",POINT (8.45669 11.87346)
165,563723584,"[6533861857, 5432184095, 6533861856, 543218409...",substation,330000;132000,outdoor,transmission,NaN,NaN,NaN,"[12.033366322222243, 11.723595733333333]",POINT (12.03337 11.72360)


In [38]:
gdf = gdf.drop(columns=['lonlat'])
gdf = gdf.drop(columns=['refs'])
gdf.to_file(f"./{countr_cod}.geojson", driver="GeoJSON")  

In [51]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world
saudi = world.query('name == "Saudi Arabia"')
yemen = world.query('name == "Yemen"')
iraq = world.query('name == "Iraq"')
oman = world.query('name == "Oman"')
uae = world.query('name == "United Arab Emirates"')
jordan = world.query('name == "Jordan"')
palestine = world.query('name == "Palestine"')
israel = world.query('name == "Israel"')
qatar = world.query('name == "Qatar"')
kuwait = world.query('name == "Kuwait"')

mena = saudi.append(yemen).append(iraq).append(oman).append(uae).append(jordan).append(palestine).append(israel).append(qatar).append(kuwait)

africa = world.query('continent == "Africa"').append(mena)
#africa.head()

In [1]:
import geoplot
nigeria = gpd.read_file('./my.geojson')
# url = 'https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/144823/africagrid20170906final.geojson'
# africa_network = gpd.read_file(url)
ax = nigeria.plot(figsize=(20, 20),alpha = 0.7)
geoplot.pointplot(nigeria, edgecolor='gray', ax=ax)
geoplot.polyplot(africa, edgecolor='gray', ax=ax)

NameError: name 'gpd' is not defined